In [1]:
push!(LOAD_PATH, "submodules")

5-element Array{ByteString,1}:
 "/Applications/Julia-0.4.5.app/Contents/Resources/julia/local/share/julia/site/v0.4"
 "/Applications/Julia-0.4.5.app/Contents/Resources/julia/share/julia/site/v0.4"      
 "/Users/rdeits/6.338/hw2"                                                           
 "/Users/rdeits/6.338/julia-mindstorms-mapping"                                      
 "submodules"                                                                        

In [8]:
using RigidBodyDynamics
using DrakeVisualizer
import Quaternions: axis, angle, qrotation
import FixedSizeArrays: Vec
import AffineTransforms: AffineTransform, tformrotate, tformtranslate
import GeometryTypes: HyperRectangle
import DataStructures: OrderedDict
using Interact

In [9]:
mechanism = rand_chain_mechanism(Float64, QuaternionFloating, Revolute{Float64}, Revolute{Float64}, Revolute{Float64})

Vertex: world (root)
  Vertex: body1, Edge: joint1
    Vertex: body2, Edge: joint2
      Vertex: body3, Edge: joint3
        Vertex: body4, Edge: joint4

In [10]:
function rotation_from_x_axis{T}(translation::Vec{3, T})
    xhat = Vec{3, T}(1, 0, 0)
    v = translation / norm(translation)
    costheta = dot(xhat, v)
    p = cross(xhat, v)
    sintheta = norm(p)
    axis = p / sintheta
    angle = atan2(sintheta, costheta)
    qrotation(convert(Vector, axis), angle)
end

function create_geometry(mechanism, box_width=0.05)
    vis_data = OrderedDict{RigidBody, Link}()
    for vertex in mechanism.toposortedTree
        if vertex.vertexData == root_body(mechanism)
            continue
        end
        geometries = [GeometryData(HyperRectangle(Vec(-0.05, -0.05, -0.05), Vec(0.1, 0.1, 0.1)))]
        for child in vertex.children
            joint = child.edgeToParentData
            joint_to_joint = mechanism.jointToJointTransforms[joint]
            q = rotation_from_x_axis(joint_to_joint.trans)
            joint_to_geometry_origin = tformrotate(axis(q), angle(q))
            geom_length = norm(joint_to_joint.trans)
            push!(geometries, GeometryData(HyperRectangle(Vec(0., -box_width/2, -box_width/2), Vec(geom_length, box_width, box_width)), joint_to_geometry_origin))
        end
        vis_data[vertex.vertexData] = Link(geometries, vertex.vertexData.frame.name)
    end
    vis_data
end

function draw_mechanism(model, vis_data, cache)
    origin_transforms = Vector{AffineTransform}()
    for body in keys(vis_data)
        T = transform_to_root(cache, body.frame)
        push!(origin_transforms, tformtranslate(convert(Vector, T.trans)) * tformrotate(axis(T.rot), angle(T.rot)))
    end
    draw(model, origin_transforms)
end


draw_mechanism (generic function with 1 method)

In [11]:
vis_data = create_geometry(mechanism)
model = Visualizer(collect(values(vis_data)));

state = MechanismState{Float64}(mechanism)
zero!(state)
cache = MechanismStateCache(mechanism, state);

In [12]:
draw_mechanism(model, vis_data, cache)

In [13]:
state = MechanismState{Float64}(mechanism)
zero!(state)

@manipulate for q1 in linspace(-pi, pi), q2 in linspace(-pi, pi), q3 in linspace(-pi, pi)
    state.q[mechanism.toposortedTree[3].edgeToParentData] = [q1]
    state.q[mechanism.toposortedTree[4].edgeToParentData] = [q2]
    state.q[mechanism.toposortedTree[5].edgeToParentData] = [q3]
    cache = MechanismStateCache(mechanism, state);
    draw_mechanism(model, vis_data, cache)
end


Interact.Slider{Float64}(Signal{Float64}(-0.06411413578754688, nactions=0),"q1",-0.06411413578754688,linspace(-3.141592653589793,3.141592653589793,50),true)

Interact.Slider{Float64}(Signal{Float64}(-0.06411413578754688, nactions=0),"q2",-0.06411413578754688,linspace(-3.141592653589793,3.141592653589793,50),true)

Interact.Slider{Float64}(Signal{Float64}(-0.06411413578754688, nactions=0),"q3",-0.06411413578754688,linspace(-3.141592653589793,3.141592653589793,50),true)

nothing